### Install and Import statements

In [1]:
# pip install mysql-connector-python

In [2]:
# pip install googletrans

In [3]:
# pip install folium

In [4]:
# Import statements to import libraries
import mysql.connector as sqcon
import pandas as pd
import numpy as np
import folium
import plotly.graph_objs as go
import plotly.express as px

### Defining the reusable functions

In [5]:
# Defining the function to create SQL connection
def create_sql_conn():
    try:
        conn = sqcon.connect(user='student', 
            password='student',
            host='cpanel.insaid.co',
            database='Capstone1')
    except sqcon.Error as e:
        print("Error Creating database connection ", e)
    return conn



# Defining the function to get the table data, it uses create SQL connection function
def getTableData(table_name):
    df = pd.DataFrame()
    try:
        conn = create_sql_conn()
        db_cursor = conn.cursor()
        db_cursor.execute('SELECT * FROM ' + table_name)
        table_rows = db_cursor.fetchall()
        df = pd.DataFrame(table_rows)
    except sqcon.Error as e:
        print("Error Reading data from DB table ", e)
    finally:
        db_cursor.close()
        conn.close()
    return df



# Defining the function to read the missing data in a dataframe
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])




# Defining the function to plot the data on Map for latitude and longitude
def plot_on_map(data_to_map):
    center = [data_to_map.iloc[0]['latitude'], data_to_map.iloc[0]['longitude']]
    map_India = folium.Map(location=center, zoom_start=4)
    #count = 0;
    for index, phone_loc in data_to_map.iterrows():
        #print(phone_loc['latitude'], franchise['longitude'])
        #count = count + 1
        location = [phone_loc['latitude'], phone_loc['longitude']]
        folium.Marker(location, popup = f'City:{phone_loc["city"]}').add_to(map_India)
    return map_India

## Week 1 activities

### Reading the data from MS SQL database

In [6]:
# Reading 'gender_age_train' data from database and then adding the required columns
gender_age_train_df = getTableData('gender_age_train')
gender_age_train_df.columns =['device_id', 'Gender', 'Age', 'Age_Group']


In [7]:
# Reading 'phone_brand_device_model' data from database and then adding the required columns
phone_brand_device_model_df = getTableData('phone_brand_device_model')
phone_brand_device_model_df.columns =['device_id', 'Brand', 'Model_Name']


### Reading the events data from the local file system

In [8]:
# Reading the events data, which is store locally
# Assuming the events_data file is in the folder named data 
events_data_df = pd.read_csv('data/events_data.csv', dtype={'device_id': np.str})
#events_data_df = pd.read_csv('data/events_data.csv') #, dtype={'device_id': np.str})

### Analyzing the data read from various datasources

In [9]:
gender_age_train_df.shape

(74645, 4)

In [10]:
gender_age_train_df.device_id.nunique()

74645

In [11]:
gender_age_train_df.Age_Group.value_counts(sort=True)

M23-26    9605
M32-38    9476
M39+      8581
M22-      7488
M29-31    7309
F33-42    5561
M27-28    5445
F23-      5050
F29-32    4628
F43+      4194
F24-26    4190
F27-28    3118
Name: Age_Group, dtype: int64

In [12]:
gender_age_train_df.describe(include='all')

,device_id,Gender,Age,Age_Group
count,7.464500e+04,74645,74645.000000,74645
unique,NaN,2,NaN,12
top,NaN,M,NaN,M23-26
freq,NaN,47904,NaN,9605
mean,-7.491354e+14,NaN,31.410342,NaN
std,5.327150e+18,NaN,9.868735,NaN
min,-9.223067e+18,NaN,1.000000,NaN
25%,-4.617367e+18,NaN,25.000000,NaN
50%,-1.841362e+16,NaN,29.000000,NaN
75%,4.636656e+18,NaN,36.000000,NaN


In [13]:
phone_brand_device_model_df.shape

(87726, 3)

In [14]:
phone_brand_device_model_df.device_id.nunique()

87726

In [15]:
phone_brand_device_model_df.describe(include='all')

,device_id,Brand,Model_Name
count,8.772600e+04,87726,87726
unique,NaN,116,1467
top,NaN,小米,红米note
freq,NaN,21645,3500
mean,-5.212557e+15,NaN,NaN
std,5.330157e+18,NaN,NaN
min,-9.223322e+18,NaN,NaN
25%,-4.623331e+18,NaN,NaN
50%,-3.109802e+16,NaN,NaN
75%,4.635194e+18,NaN,NaN


In [16]:
events_data_df.shape

(3252950, 7)

In [17]:
events_data_df.device_id.nunique()

60865

In [18]:
events_data_df.describe(include='all')

,event_id,device_id,timestamp,longitude,latitude,city,state
count,3.252950e+06,3252497,3252950,3.252527e+06,3.252527e+06,3252950,3252573
unique,NaN,60865,588126,NaN,NaN,933,32
top,NaN,5536513450525271144,2016-05-03 10:00:03,NaN,NaN,Delhi,Delhi
freq,NaN,33426,43,NaN,NaN,744276,751733
mean,1.626476e+06,NaN,NaN,7.815868e+01,2.168851e+01,NaN,NaN
std,9.390459e+05,NaN,NaN,4.235570e+00,5.789111e+00,NaN,NaN
min,1.000000e+00,NaN,NaN,1.256700e+01,8.190110e+00,NaN,NaN
25%,8.132382e+05,NaN,NaN,7.583543e+01,1.780171e+01,NaN,NaN
50%,1.626476e+06,NaN,NaN,7.726814e+01,2.216454e+01,NaN,NaN
75%,2.439713e+06,NaN,NaN,8.031916e+01,2.868278e+01,NaN,NaN


### Getting Missing Data from datasources

In [19]:
missing_data(gender_age_train_df)

,Total,Percent
Age_Group,0,0.0
Age,0,0.0
Gender,0,0.0
device_id,0,0.0


In [20]:
missing_data(phone_brand_device_model_df)

,Total,Percent
Model_Name,0,0.0
Brand,0,0.0
device_id,0,0.0


In [21]:
missing_data(events_data_df)

,Total,Percent
device_id,453,0.013926
latitude,423,0.013004
longitude,423,0.013004
state,377,0.011589
city,0,0.000000
timestamp,0,0.000000
event_id,0,0.000000


#### Checking unique values in each column for various data frames

In [22]:
gender_age_train_df.nunique()

device_id    74645
Gender           2
Age             85
Age_Group       12
dtype: int64

In [23]:
phone_brand_device_model_df.nunique()

device_id     87726
Brand           116
Model_Name     1467
dtype: int64

In [24]:
events_data_df.nunique()

event_id     3252950
device_id      60865
timestamp     588126
longitude      60095
latitude       60185
city             933
state             32
dtype: int64

### Analysis on Missing data from events data

In [25]:
events_data_df.value_counts()

event_id  device_id             timestamp            longitude  latitude   city       state        
3252950   -4383315854394153449  2016-05-07 23:20:08  79.172943  13.017156  Velluru    TamilNadu        1
1084334   2237921149420055154   2016-05-03 09:21:33  75.910331  22.768655  Indore     MadhyaPradesh    1
1084332   969634985145229996    2016-05-03 09:27:53  80.293548  13.188370  Chennai    TamilNadu        1
1084331   -4988638036225877537  2016-05-03 09:11:19  77.613337  13.056750  Bangalore  Karnataka        1
1084330   3626734845163810822   2016-05-03 09:10:54  89.530936  26.362451  KochBihar  WestBengal       1
                                                                                                      ..
2168612   4891500938696877919   2016-05-06 21:02:16  73.914889  18.587531  Pune       Maharashtra      1
2168611   5025286536641049649   2016-05-06 21:07:00  73.939161  18.561320  Pune       Maharashtra      1
2168610   -3293430962013612634  2016-05-06 21:01:48  88.3989

#### Checking missing data of various columns based on City as City column does not have any missing values

In [26]:
events_data_df[events_data_df.latitude.isnull()]['city'].value_counts()

Jaipur           66
Indore           63
Delhi            63
Chennai          63
Pune             63
Visakhapatnam    63
Bagaha           14
Moga             14
Araria           14
Name: city, dtype: int64

In [27]:
events_data_df[events_data_df.longitude.isnull()]['city'].value_counts()

Jaipur           66
Indore           63
Delhi            63
Chennai          63
Pune             63
Visakhapatnam    63
Bagaha           14
Moga             14
Araria           14
Name: city, dtype: int64

In [28]:
events_data_df[events_data_df.state.isnull()]['city'].value_counts()

Delhi            64
Pune             63
Indore           51
Jaipur           49
Chennai          47
Visakhapatnam    47
Gangarampur      26
Arambagh         16
Channapatna      14
Name: city, dtype: int64

In [29]:
events_data_df[events_data_df.device_id.isnull()]['city'].value_counts()

Jaipur           81
Pune             72
Delhi            69
Visakhapatnam    69
Chennai          63
Indore           51
Bardoli          16
Jetpur           16
Hoshiarpur       16
Name: city, dtype: int64

#### Checking common Unique device id's between each data frames

In [30]:
genderage_device_set = set(gender_age_train_df['device_id'].astype(str).unique())
phone_brand_device_set = set(phone_brand_device_model_df['device_id'].astype(str).unique())
events_device_set = set(events_data_df['device_id'].unique())

int_pb_ga_set = genderage_device_set.intersection(phone_brand_device_set)
common_devices_set = events_device_set.intersection(int_pb_ga_set)
#common_devices_set = genderage_device_set.intersection(phone_brand_device_set).intersection(events_device_set)

In [31]:
print("Gender Age count : {0}\nPhone Brand Count : {1}\nEvents Device Count : {2}".format(len(genderage_device_set), len(phone_brand_device_set), len(events_device_set)))
print("Common devices between Gender Age and Phone Brand Datasets is : ", len(int_pb_ga_set))
print("Common devices between all 3 Datasets is : ", len(common_devices_set))

Gender Age count : 74645
Phone Brand Count : 87726
Events Device Count : 60866
Common devices between Gender Age and Phone Brand Datasets is :  74617
Common devices between all 3 Datasets is :  60865


## Week 2 activities

### Convert the Chinese names of brand and Model to respective English names

In [32]:
brand_list_chinese = list(phone_brand_device_model_df['Brand'].unique())
model_list_chinese = list(phone_brand_device_model_df['Model_Name'].unique())

In [33]:
# Model and Brand names are converted to English names using Google Translator manually
brand_list_eng = ['vivo', 'Xiaomi', 'OPPO', 'Samsung', 'Coolpad', 'Lenovo', 'Huawei', 'Qiku', 'Meizu', 'Phixun', 'China Mobile', 'HTC', 'Tianyu', 'Extreme Bao', 'LG', 'Oberxin', 'Yumi', 'ZUK', 'Nubia', 'HP', 'Nibiru', 'Meitu', 'Xiangmi', 'Motorola', 'Mengmi', 'Hammer', 'InFocus', 'LeTV', 'Hisense', 'Bai Li Feng', 'OnePlus', 'Yuxin', 'Haier', 'Cubbie', 'Newman', 'Bird', 'Dove', 'Lingyun', 'TCL', 'Copper', 'Aipel', 'LOGO', 'green onion', 'fruit rice', 'ASUS', 'Onda', 'Aiyouni', 'Konka', 'Yougo', 'Banghua', 'Cyber Yuhua', 'Black Rice', 'Lovme', 'Pioneer', 'E School', 'Shenzhou', 'Nokia', 'Pnair', 'Candy Gourd', 'Yitong', 'Ouxin', 'Mickey', 'Coolby Rubik\'s Cube', 'Blue Devils', 'Little Poplar', 'Bairfeng', 'Nuo Mi', 'Mi Song', 'E People E Book', 'Simi', 'BigQ', 'Taipower', 'Philips', 'Weimi', 'Daxian', 'Changhong', 'Vitu', 'Green Orange', 'Originally', 'Xia Mi', 'Amoi', 'Wei Mi', 'Bajia', 'SUGAR', 'Ouqi', 'Century Star', 'Zhimei', 'Obi', 'Kivu', 'Femtosecond', 'Desai', 'Yipai', 'Google', 'Venus Digital', 'Guangxin', 'Noah Xin', 'MIL', 'White Rice', 'Coke', 'Baojiexun', 'Youyu', 'First Cloud', 'Rimei', 'Rigao', 'Wopfung', 'Mole', 'Fresh Rice', 'Kailyton', 'Weibi', 'Ouwo', 'Fengmi', 'Hengyufeng', 'Ox', 'Siemens', 'Orlandi', 'PPTV']
model_list_eng = ['Y13','V183','R7s','A368t','Redmi Note2','Redmi Note3','S5.1','Galaxy Grand Prime','MI 2S','Galaxy Note 2 ','R1C','Great God F1Plus','note top version','Galaxy Mega 2','Star No. 1','Galaxy S4','MI 3','Find 7','C8816', ' Redmi note','X6 L','Galaxy Win','Youth Edition','MX3','Y35','G520-5000','Honor 4A','Charming Blue Note 2','Y560-CL00 ','Honor 7i', '3000','Honor Play 4C','C730Lw','Redmi 2A','A1','One','Galaxy J7','R3','MX2', ' Honor Play 5','Honor 7','MI 4','One M8','Redmi 1S','Maimang 4','Galaxy S6 Edge','A3800d','P780','Ascend G7' ,'Honor 6','Xperia SL','Y17T','Honor Play 4X','Ascend P7','Galaxy Grand 2','Honor 3X Play Edition','Honor X2','Ascend P8' ,'R2017','Charm Blue NOTE','Mate 2','M8','T87+','X710L','A890e','R7','X5M','MX5','Galaxy Mega 5.8', 'L81','Xperia Z1','Honor 6 Plus','X5Max+','Galaxy J5','Galaxy Note 4','A31','GN706L','G3','P8 Lite','Sensation XE with Beats Audio Z715e','A880','Honor+','X5L', '8720L','IVO 6655','R7 Plus','Honor Play 4','MX4','Y635-TL00', 'Galaxy S3','X3L', '3','R807','Golden Fighter A8','One X','Xiaomi note','R6007','Galaxy S2 Plus ','R1','Honor 3C','Mi 4C','R823T','Mate 7','MI 2','F2','Galaxy S5','Xperia TX','Galaxy Trend DUOS 2' ,'Z1','Y11IT','Redmi Note Enhanced Edition','Tianjian W900','Ascend G700T','Galaxy Note 3','Galaxy R','F301','Honor Play 5X', 'N1 Mini','Find 5','Redmi 2','X5Pro','X5V','A11','MI pad','Z7 Mini','Galaxy Note 5','MediaPad M2-801W' ,'Galaxy Grand Neo Plus','Ascend GX1','R817','X3T','Galaxy E7','X817','GN700W','Y20T','A680','Y27', '5891', '大神F1','UIMI3','Compaq 7','G628','IVO 6622','B199','Galaxy Trend 3','X1ST','S7568I','C8818','Y622', ' Charm Blue 2','Mate 7 Youth Edition', '1107','Galaxy S2','Le Meng K3 Note','Galaxy S4 Zoom','Galaxy Ace Dear','Mars One','Le Meng K3 ','Ascend P1','R831T','M9','U9508','X5Max', '2','X8888','Lenovo Gold Fighter S8','S60t','Great God F2','Xplay3S' ,'R1S','Y60-C1','Nexus 5','UIMI4','R809T','Q507T','Moto X Style','A788t','Charm Blue','Y13iL','Honor 6 plus','G610S','N1T','X1 7.0','Y613F','Galaxy A7','N1','Y23L','Great God F2 Full HD Edition','Inspiration XL','Ultimate Edition','Y927','Z9 mini','X3S','Y15','R831S','R815T','R819T','Galaxy Core Prime','Butterfly','Galaxy Style DUOS','Xperia S','Galaxy Note 3 Lite','P6','S7','Galaxy S6','Nut Phone','M210','S7I' ,'A51','X907','Redmi','Y33','Super Phone 1 Pro','Galaxy Grand DUOS','Xperia Z5','G620-L75','Enjoy 5','Xplay ','MX4 Pro','Galaxy S6 Edge+', '8185','G3588V','Y937','T528t','Galaxy A8','N3','G610T','S7T','V5 Max' ,'MI One Plus','S90-t','Xperia Z3 mini', '8105', '7270-W00','M8St','Desire 820','A750','S898t','Memorial Blue metal ','P970','Galaxy Mega 6.3', '5890','Z9 Max','PRO5','X5SL','Y29L','E602T','GN150', '8190Q','E620M', ' Super phone 1', '8702D','Galaxy Grand','Galaxy Note','U960E','Grand X','Xperia arc S','T708','X','T50','GN128', 'Super Phone 1s','L700T','Galaxy A5','F103S', '5951','H1','A820','G7200','Honor U8860','Button','LT01','Y11 ','Y635','Honor Play Tablet T1','Xperia Neo','G7 Plus','Y13L','HT-I860', '8056','EVO 3D X515m','Red Pepper X1', 'Galaxy A3','Spring Thunder HD','Galaxy S 2 HD LTE E120S','L5Pro','ivvi Xiaoi','Honor Play 4C Sports Edition','Maimang 3','Ascend G700','Xiaoxian 2','Nexus 4','R827T', 'G520-0000','Ascend P6','ELIFE E3T','Daqi 2', '5950','Q519T','Galaxy Tab P6800','A398t','Fengshang','Z5S', ' Y511-T00','MI One C1', '8085q','A3900','M4','L1 mini','V188S','大神X7','Grand X Quad','A320t','G2' ,'Galaxy Core Advance','Y28L','Y22IL','Little Apple','SHV-E210L','Lewan','F103','Y928','Galaxy Ace 3','R821T', ' Great God Note3','MI 2A','Moto X','Y321C','Xperia Z2','I9118', '8750','K860i','G730-C00','A708t','A670t', ' S658t','Galaxy On7','Desire 820 Mini','Galaxy Ace','T1','A33', '603e','LT416','G11','S7568','T528w', '6607' ,'Momada 3N', '8730L','M2','Y11IW','W2015','Fengshang Pro','R830','ELIFE S5.5','Q505T', '9190l', ' Galaxy Note Edge','Touch 2C','Red Pepper','X3F','Y913','Galaxy Trend DUOS','Optimus G', '7296','R5','M512','Galaxy On5' , '7105','Galaxy S3 Neo+','G520-T10','VIBE X','Mate 8','S1','MI 1S', '5263','S2y','A369','Grand Memo 2','G750-T20','Y923','M032','R8205','A688t','Honor 3X','Samsung big foot','Touch 2 ','Galaxy Premier','Y37','U956','S810t','HN3-U01','S650','L128','IPH8','Honor 3C Play Edition', '1100', ' S5830I','X6 Plus D','G606','B9388','Desire 826','A5800-D','Galaxy Core Lite','MI pad 2','A360t','SM-T705C', 'G4','ono','Find','Galaxy Core Max','ivvi Small Bone Pro','A53','G718C','Red Pepper XM','V5S', '3007','D800', 'X6 Plus','S12','I779','Y1','Yuanhang 3','My Prague','Y18L','Blade S6 Lux','H1S','GN715','metal Standard Edition' ,'U879','Desire 7088','U930HD','Galaxy Note 10.1','G3818','Changxiang 5S','U701','X6 D','MediaPad M2-803L','Blade A1' ,'Galaxy Nexus','Desire 610t','Y19T','Red Pepper Note','Fengshang Max','Galaxy S L', '2016 Version Galaxy A9','Galaxy S3 Mini','GN152', 'Desire 816','VIBE X2','A516','S7898','Red Bull V5','Moto X Pro','Y13T', '2016 Galaxy A5','S11T','VIBE Z2', ' A7', '7270','S6','Xperia C', '7295','锋尚2','V4','M5','H5','R833T','Grand S 2','G510 ','C8817E', '5263S','Y22L','S8 20','Y51','Golden Steel','T329t', '7298D','Wildfire S','Galaxy J3','iPh-800','C8815','P335M','S5.5L', '1105', '8729','T328w','Xperia Z','W2014','Galaxy Mega Plus','G660-L075', '5200S','Y613','Mate S','A298t', 'Galaxy Tab 3 8.0','T20L','U880F1', '2016 Galaxy A7','X8T','Two','N1 max','Red Rice 3','R820','Little Pepper X3' ,'L920','One S','Ulike2','A376', '8029','U707T','Galaxy Win Pro','Z5S Mini','ZenFone Max','E601M','GN708T','Xperia C3', 'I8268','Galaxy Tab 3 10.1','ZenFone 2','Honor Tablet T1-823L','Y600D','S2L','Galaxy Tab 2 P3100', '8122','Y518-T00', ' V819 3G','D2-M','A580','U3','S720','Xperia acro S','X3V','GN151','X6', '5891Q', '5956','P316L ',':UOOGOUT1','Y516','C8816D', '7269','Galaxy S7 Edge','U11','SOP M3','A828t','Y3T','大神Note', '7295C' ,'A850','U705T','A390','Galaxy Tab P3108','A789','E1T','A2800d', '7295A Youth Edition','MI 2C','M031','C8813', 'A199', '795+','E6', '802w','Xperia T','D500', '5219','R813T','D350', '8190', '7231','A678t', 'Galaxy Note 8.0','A380t','A820t', '7372','X1','E1','X1S', '8195','MT1-U06','Y600-U00','S890', 'HS-U978','H7','Y320','Xperia P','Galaxy Ace 2','S920','Q705U','U817','T2','S9','V719', ' A356','EG939','A388t','R811','V1','L22', '8720','S850t','S3+','V182','M100','P770','iSuper S2 ','A798t','T9','C8813Q','EG978 ','Galaxy S Advance', '7620l', '8702','MI 4S', '8705','T29','Y17W','K68w','Xshot','Ascend G6','G730-T00 ','GN700T','Big Q Note','P880','X50TS','M811','X68T','A390t','X6666','Y618-T00','ZenFone 5','IdeaTab A5000 -E','Galaxy Ace Plus','M1','Galaxy Tab S T705C','Galaxy Tab 4 T231','G730-U00','U809','MI 5','V8','VIBE Z ','XL','MOMO9','M310','Galaxy Core Mini','GN5001S','Momada','Y330','Little Pepper M2','IVO 6600','Golden Fighter Note8' ,'Galaxy Trend 2','Mate', '7295+', '7320','U930','Y300','U7','A660','T3','P308L','U969','Xperia Z Ultra','T91','Hyunying S+','Galaxy Tab 7.0 Plus','M701','A658t','Moto G','P331M','P700i','N798+', '7235', 'Galaxy S4 Active','E7','Xperia Z3','S720i','TAB 2 A7-30','P1', '8085n','S5300','V987','Grand Memo','K1 ','L7','P520L','P700','Galaxy S4 Mini','Fenghua 3','Xperia SP','V70','MT2-C00','Xperia ion', '7268', '5892','G521','M6-M7', ' TALK 7X Quad Core','i9','V889D','iSuper S1', '7236','T9108','MM1101','L810','Xperia T2 Ultra','U960S3','Touch 3', '7230','Galaxy Gio','U9815', '8012','A630t','V818','Galaxy J1','Galaxy Fame','A800','T958','A820e','A278t' ,'N1W','Y500','U960S', '9976A','GN100','X805', '8295M','C8817L','A505e', '8198T','GN137','U51GT-W' ,'K900','ME371MG','A520','J738M','A766', '5860A','R9','S696', '9180','S868t','TALK 9X','Optimus LTE' ,'G615-U10','A830','Galaxy Core 4G','Y511-U00','V185', '2016 version Galaxy J7','Y80D','天鑑T1','G716','Galaxy W','R830S','Y22','A850+','U795+','R8000','E5','GN9002','E6T','Galaxy J', '5217','Z9','Saturn One','W999','XT615','Y627','Vibe Z2 Pro','Maimang 3S', '8079','Q802T','W2013','BF T18','N910C','E912S ', '8295','L9','A770e','Q501U','SCH-I939','G6-U00','L1','E3','A698t','A765e','Xperia Z3+ Dual ','A3000-H','Y623','G717C', 'Pegasus','T329d','T8620','S9T', '3S','ivvi K1 mini','S5831I','Lenovo VIBE X2', '7296S','U966','A785e', ' T328t','T7','Mighty 3','P8max','E70-T','E8','R9 Plus','P306C','One E8','Q503U','Galaxy S2 Epic 4G Touch ','P9', '201','Droid Ultra','T80','M812C','V955', '3005','Red Pepper Willpower Plus','L5 Plus','Galaxy Alpha', ' S938t','XT1060','U950','A2580','S899t','VS980','U980','C8650','S850','U2','Vibe Shot','Desire 626', ' X2','Z7 Max','Galaxy Mini','A750e','T619+','S7566','Little Pepper 9','Galaxy Core 2','R805','GN305','C8812','A760','Grand View 4','A399','Super Phone Max','Galaxy K Zoom','R5s plus','P332U','N910U','Star 2','Blade S6', ' V956','Thunder Fighter','EG970','S5.1Pro','MI 1','X80HD','A858t','V526','Y628','G610C','R801','Y3100' ,'Touch 3c','威武3C','F303','One max', '5360','A560', '8021','Galaxy Golden','I9050', '7i','Qingcheng L3', 'M20-T','E613M','G730-L075','Galaxy S Plus', '8085','Z9 mini Elite Edition','Little Pepper S1', '8721','S686','S858t' ,'P705','L8','Little Pepper 5','G3609','E75T','Optimus G Pro','Galaxy Tab 3 Lite T111','R817T','T03 Fengzhi Edition','N909 ','乐玩2C','GN708W','C630Lw','VT898','One M9+','Nexus S','A7-30','S930','P70','C230w','Desire 616', '7260','AxonMini', '8712S','GT7','Galaxy Tab 4 T331C','One E9','Momada 3S','N1-Y','Y31','V5 ','Lemon K31','Golden Fighter S8 Play Edition','SM-T325','Galaxy Trend','IdeaTab A2207A-H','A318t','U807','XT910','A706' ,'A330e','D520','K860','A8-50','SM-T2558', '8295c','ELIFE E6 mini','A630','Grand Era','TL1000','Z5 ','S5000','C8813DQ','T9508','R2010','Q201T','N821','idol X','DROID RAZR','G6-C00','ELIFE E7 Mini','A5','Ascend D2','V967S','X5','GN168t','YOGA Tablet B6000','G3819D' ,'Galaxy Tab 2 P5110','V6','Y320T','A288t','Touch V','Fashion Phone','A628t','OKU3','I8258','Galaxy Tab 3 7.0', ' Defy+','P301M', '5872','V9', '8732', '8070','Galaxy Y', '7060','S2','GT-I8160','T967','A3300-T ','T5','Tab2 A7-10F','A308t','Galaxy Tab Pro T321','V975s Core4','Desire S','L168','R850','GN705T', '7251', 'M601','U5482','S90-u','Xperia U','U880','D330','C986t','EV5','T528d','G630','A656','S660' ,'T60','Y325','Q7','I95', '8089','Galaxy S', '5832','G525','Desire 310','EG966','V10','大神1s ','GN135','S3','M10-M','A378t','A889','s820e', '5860S', '8076D', '8017','Butterfly s','M6','Xperia ZR', '608t','GN810','N986','Blade C', '7019','N828','S880i','M030','G616- L076','A208t','GN139','VT898S','U939','Q301C','Galaxy Ace DUOS','A269i','GN878','One M8 Eye','A238t','VT868' ,'A398t+','Q701C','TS1018','N818','T20', '8707','S8-701u','Desire 5088', '8017-T00','M8x','T327w', 'GN105','E600M','Xperia ZL', '606w','MK260','memo 5S','Desire 609d','A3','Desire D516d','T8830Pro','HT-I717', 'A6800','I630T','T35', '8713','Galaxy Tab S T800','E621T','Galaxy Grand Max','Allure L3C','t789','U701T','Nexus 7' , '5218S','L70','M3','Q302C','U8825D','V919','One ME','Galaxy Infinite','Y220T','IVO 6688','Y610-U00', '5216D','Little Pepper 4','T820','X920e','L820','S939','Xperia Z1 Colorful Edition','T9003','Weike','T21','V95' ,'A860e','V188','S8-701w','S5Q','One M9','voga v1','MM1103','M535', '5261','i1','idol3', ' G5','S5','T12','A2860','Golden Fighter Youth Edition','P8', '802d','Red Pepper NX Plus','A51kc','T004','BF A500', 'T1-821w','Sensation Z7 10E','Xiao Xian 3','Little Stars','M5 Plus','Galaxy Note 10.1 2014 Edition P601','Galaxy S DUOS','A355e','A30','TL500','Z5 Mini' ,'BM001','A690','A68e','W619','GN106','GT-N8000','EG971','N880F','J320T','EG936D','Desire D516t','T929 ','I8250','A338t','T327t','N881F','GN180', '5316', '5930','S880','MC002','U819','JX8166','A300t', '8717', '9976D','Desire D316d','A805e','KDOOR X5-Y12','大器3','S960','X7','Desire HD','E602M','H2' , '78P01','A588t','T619', '8150','LT02','M1L','U8', '8712', '2C','M201S','T90','M20-M','SM-T2556','M3S','S6-NT1','A3600d','X98 3G','Little Pepper 6','A606','Q801L','P318L', ' S860e','T9600','T890','Galaxy Tab P7500','King Kong II','F105','L108','A768t','D557','W800','VIBE P1','L100' ,'R2','ZTE Yuanhang 3','C03','Axon Tianji','OKA51','A385e','Galaxy Tab 2 P5100','N880E','MediaPad 10 Link','XT1033', ' EG980', '210R','U86','Doeasy E700','Q501T','Y310-T10','Y310-5000','S6T','s800','GN108','K88L','U1 ','Z4', '8076', '5311','L930i','Xperia M2','P2','L820c','VT888','U81t','Xperia ZR','I630M','C8813D ','青洋3','Xperia ray','GN9004','Galaxy Grand 2 LTE', '5876','C8812+','D516w','D360','Little Pepper 3','G629', 'P500M','GN800','N2','DM','I639T','PHAB Plus','Galaxy Tab 3 Lite T110','U968','Sonic+','P302C','Nexus 6P', '7605','Y70-C','U808','T703','K23','旋影90w','V931','Little Pepper 7','W700','H3','Tab2A7-20F ', '8736','Mos1','S8-50', '5216s','W88','Ascend G330C','BM002','Drunk Enjoy','MAX3','L5 mini','Hungry for HD ','A358t','N919D', '5879','E913','V981','Little pepper M 3','Galaxy Tab S T805C','Desire V','I635T','E99','X6088','Optimus G Pro 2','A500', '5310', '7275','P501M', 'P518L','E260-T','U818','P502U','I900','U880S','Pegasus 3', '7061','BM002-G5','P360W','P21', 'E968','HT-I857','Y523-L176','Leading Flagship 3','K80','Galaxy Ace 4','L82','Moto X 3','Little Pepper X4', ' V880','i600','U889','F320L','K012','U55GT','Xperia Z5 Premium','A375e', '302U','T00G','L823','C666t', ' EG929','MediaPad M1','W6618','A590','a218t','E956Q','S300','A3500','E50-T','GN705','BM003','A780','D1','Q3C','A360e','P305','F1','LT988', '5313S','Red Pepper Note3','Galaxy S7','Xplay5' ,'Desire 816x','S668t','T9608','IdeaTab A1010-T','W700+', '5200','F240S','M15','T928','Q805T','U970', ' A396','S90-e','天鑑W808','J636D+','V923','T13','P51','X10i','E820','GN5002','G9','P9 Plus ','A328t', '8715','U880E','天鑑W900S','One A9', '8722','V889M','T780+','N919','Galaxy Core','U793', 'L855','T9500','P3','Daiguan Platinum','S560','Xperia J','A3890','G610M','Charm Blue Note3','Hol-U10','Q509T' ,'XT928','X18','A630e','MM1104','I7','A380e','A65','U25GT','Razr V 锋芒','T6','IVO 6666','Z7 ','S8-303L', '9300-M9','V983','G730-U30','V701s Core4','A366t','V965T','ME7510KG','W2016','One X9', 'T328d','YOGA Tablet B8000','A100','C986t+','ZenFone Selfie','Mars One Exploration Edition','P596','S838M','IdeaTab A1000-T','G309T', 'A780e','DROID RAZR M','VIBE X2 Pro','Moto X Pole','A395e','Gigaset ME','Ascend D1','Galaxy Tab P1000','Find 3','U960' ,'G620-L72','T918','U9 36','I699I','E622M','X80h','B8080-F','Magic Mirror X5','Moto Turbo','A1020','T860','S870e','Little Octopus', ' Z1988','X98 Air 3G','King 7','King 7s','T621','Y320C','M228-M51','I6','Galaxy S DUOS 2','Lemon 3', 'T89-2','m310','H6','Lemon X3','Pegasus 2 Plus','Galaxy Tab S T705','IVO 8800','U59GT']

phone_brands = {'brand_chinese':brand_list_chinese, 'brand':brand_list_eng}
phone_brands_df=pd.DataFrame.from_dict(phone_brands)
phone_brands_df

phone_models = {'Model_Name_Chinese':model_list_chinese, 'model_name':model_list_eng}
phone_models_df=pd.DataFrame.from_dict(phone_models)
phone_models_df

phone_brand_device_model_df = pd.merge(phone_brand_device_model_df, phone_brands_df, how ='left', left_on='Brand', right_on='brand_chinese').drop(['brand_chinese', 'Brand'], 1)
phone_brand_device_model_df = pd.merge(phone_brand_device_model_df, phone_models_df, how ='left', left_on='Model_Name', right_on='Model_Name_Chinese').drop(['Model_Name', 'Model_Name_Chinese'], 1)



In [34]:
# Validating top 10 rows after Brand and Model name update
phone_brand_device_model_df.head(10)

,device_id,brand,model_name
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,Xiaomi,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,Samsung,A368t
4,-2015528097870762664,Xiaomi,Redmi Note2
5,-8896952452425303597,Xiaomi,Redmi Note3
6,-2634171391322875113,Xiaomi,S5.1
7,-2526386806972133373,Samsung,Galaxy Grand Prime
8,1427934937997164515,Xiaomi,MI 2S
9,2866484947180243791,Samsung,Galaxy Note 2


### Replace the NaN values

##### Considering city as the column for taking all the modes as  this column has no missing values

##### Replacing the missing Latitude and Longitude for the records with the maximum occured (mode) latitude and longitude for a city

In [35]:
# Creating a list of all cities where the latitude/longitude is missing 
missing_ll_city_list = events_data_df[events_data_df.latitude.isnull()]['city'].unique()

In [36]:
print(missing_ll_city_list)

['Delhi' 'Indore' 'Visakhapatnam' 'Chennai' 'Pune' 'Jaipur' 'Araria'
 'Moga' 'Bagaha']


In [37]:
# Creating a dictionary of city and modes of latitude and longitude
city_ll_mode_dict = {}

for city in missing_ll_city_list:
    mode_latitude = (events_data_df.loc[events_data_df['city'] == city]['latitude']).mode()[0]
    mode_longitude = (events_data_df.loc[events_data_df['city'] == city]['longitude']).mode()[0]
    city_ll_mode_dict[city] = [mode_latitude, mode_longitude]
    
print(city_ll_mode_dict)

{'Delhi': [28.749820000000003, 77.29399099999998], 'Indore': [22.809817000000002, 75.89546299999998], 'Visakhapatnam': [17.829748000000006, 83.326714], 'Chennai': [13.128174, 80.359796], 'Pune': [18.616127, 73.926254], 'Jaipur': [26.949746, 75.89405699999998], 'Araria': [26.211919, 87.570736], 'Moga': [30.904178, 75.268748], 'Bagaha': [27.17739700000001, 84.1409]}


In [38]:
# Updating the Latitude and Longitude with the value of mode latitude/logitude for the city
for city in city_ll_mode_dict:
    events_data_df.loc[((events_data_df['city'] == city) & (events_data_df['latitude'].isnull())), 'latitude'] = float(city_ll_mode_dict[city][0])
    events_data_df.loc[((events_data_df['city'] == city) & (events_data_df['longitude'].isnull())), 'longitude'] = float(city_ll_mode_dict[city][1])

##### Replacing the missing value of State for the records with the existing value of state for a city

In [39]:
# Creating a list of all cities where the state is missing
missing_state_city_list = events_data_df[events_data_df.state.isnull()]['city'].unique()

In [40]:
# Creating a dictionary of city and states
city_state_dict = {}

for city in missing_state_city_list:
    city_state_dict[city] = (events_data_df.loc[(events_data_df['city'] == city) & (~events_data_df['state'].isnull())]['state']).reset_index(drop=True)[0]
    
print(city_state_dict)

{'Pune': 'Maharashtra', 'Visakhapatnam': 'AndhraPradesh', 'Indore': 'MadhyaPradesh', 'Chennai': 'TamilNadu', 'Delhi': 'Delhi', 'Channapatna': 'Karnataka', 'Jaipur': 'Rajasthan', 'Gangarampur': 'WestBengal', 'Arambagh': 'WestBengal'}


In [41]:
# Updating the state for the missing values of state for a city
for city in city_state_dict:
    print("The city and state are {} and {}".format(city, str(city_state_dict[city])))
    events_data_df.loc[((events_data_df['city'] == city) & (events_data_df['state'].isnull())), 'state'] = str(city_state_dict[city])

The city and state are Pune and Maharashtra
The city and state are Visakhapatnam and AndhraPradesh
The city and state are Indore and MadhyaPradesh
The city and state are Chennai and TamilNadu
The city and state are Delhi and Delhi
The city and state are Channapatna and Karnataka
The city and state are Jaipur and Rajasthan
The city and state are Gangarampur and WestBengal
The city and state are Arambagh and WestBengal


In [42]:
#events_data_df[joint_lat_long] = events_data_df['latitude'].to_str + '-' + events_data_df['longitude'].str

In [43]:
# Adding a temporary column by combining latitude and longitude
events_data_df['joint_lat_long'] = [''.join([str(x), '-', str(y)]) for x, y in zip(events_data_df['latitude'], events_data_df['longitude'])]

In [44]:
events_data_df.head()

,event_id,device_id,timestamp,longitude,latitude,city,state,joint_lat_long
0,2765368,2973347786994914304,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi,28.73014-77.22567599999998
1,2955066,4734221357723754978,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal,22.660325-88.388361
2,605968,-3264499652692492784,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi,28.757906-77.25680899999998
3,448114,5731369272434022440,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu,13.153332-80.34361299999998
4,665740,338888002570799963,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand,23.842609-85.997745


In [45]:
events_data_df.nunique()

event_id          3252950
device_id           60865
timestamp          588126
longitude           60095
latitude            60185
city                  933
state                  32
joint_lat_long      60869
dtype: int64

##### Replacing the missing device_id for the records with the maximum used (mode) location for a device

In [46]:
missing_device_id_ll_list = events_data_df[events_data_df.device_id.isnull()]['joint_lat_long'].unique()

In [47]:
print(missing_device_id_ll_list)

['22.814519-75.88295600000002' '26.948689-75.888487' '26.960796-75.846007'
 '22.777781-75.92333199999999' '31.561746999999997-75.99255099999998'
 '18.628057-73.862756' '18.566925-73.86016500000002' '17.805195-83.357991'
 '28.719966000000003-77.29248100000002' '18.614812-73.92649899999998'
 '13.153332-80.34361299999998' '13.149176-80.30927199999998'
 '28.721053-77.27481399999998' '17.752819-83.371738' '22.817526-75.95805'
 '17.822906-83.342711' '26.95399-75.836167' '13.189053-80.33543499999998'
 '21.194283-73.16934499999998' '21.790693-70.686387'
 '28.728888-77.30315300000002']


In [48]:
# Creating a dictionary of lat_long and device_id
ll_device_id_dict = {}

for ll in missing_device_id_ll_list:
    ll_device_id_dict[ll] = (events_data_df.loc[events_data_df['joint_lat_long'] == ll]['device_id']).mode()[0]
     
print(ll_device_id_dict)

{'22.814519-75.88295600000002': '3132861355949762783', '26.948689-75.888487': '-917725110041294385', '26.960796-75.846007': '-8460337302442785825', '22.777781-75.92333199999999': '7597735768595924821', '31.561746999999997-75.99255099999998': '1750778632182066836', '18.628057-73.862756': '9027085705222633039', '18.566925-73.86016500000002': '8539870903150472243', '17.805195-83.357991': '6844192269280790123', '28.719966000000003-77.29248100000002': '5098778421671837341', '18.614812-73.92649899999998': '177158571165078698', '13.153332-80.34361299999998': '5731369272434022440', '13.149176-80.30927199999998': '7662139143719919126', '28.721053-77.27481399999998': '3724654925765159056', '17.752819-83.371738': '-3130903177076173323', '22.817526-75.95805': '-2399250255599832969', '17.822906-83.342711': '6619714762103551896', '26.95399-75.836167': '3562355872640618177', '13.189053-80.33543499999998': '7983871429718484702', '21.194283-73.16934499999998': '-8215770519233685145', '21.790693-70.6863

In [49]:
for ll in ll_device_id_dict:
    print("The ll and device ids are {} and {}".format(ll, str(ll_device_id_dict[ll])))
    events_data_df.loc[((events_data_df['joint_lat_long'] == ll) & (events_data_df['device_id'].isnull())), 'device_id'] = str(ll_device_id_dict[ll])

The ll and device ids are 22.814519-75.88295600000002 and 3132861355949762783
The ll and device ids are 26.948689-75.888487 and -917725110041294385
The ll and device ids are 26.960796-75.846007 and -8460337302442785825
The ll and device ids are 22.777781-75.92333199999999 and 7597735768595924821
The ll and device ids are 31.561746999999997-75.99255099999998 and 1750778632182066836
The ll and device ids are 18.628057-73.862756 and 9027085705222633039
The ll and device ids are 18.566925-73.86016500000002 and 8539870903150472243
The ll and device ids are 17.805195-83.357991 and 6844192269280790123
The ll and device ids are 28.719966000000003-77.29248100000002 and 5098778421671837341
The ll and device ids are 18.614812-73.92649899999998 and 177158571165078698
The ll and device ids are 13.153332-80.34361299999998 and 5731369272434022440
The ll and device ids are 13.149176-80.30927199999998 and 7662139143719919126
The ll and device ids are 28.721053-77.27481399999998 and 3724654925765159056


In [50]:
#Dropping the temporary column
events_data_df = events_data_df.drop(['joint_lat_long'], axis = 1)

In [51]:
#check the missing data
missing_data(events_data_df)

,Total,Percent
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0
event_id,0,0.0


#### Analysis on Latitude and Longitude


The latitude and longitude for India are in the range:<br/>
<b>Longitude</b> : 68.7 - 97.25 (West  -> East)<br/>
<b>Latitude</b>   : 08.4 - 37.60 (South -> North) <br/>
If we can find the latitude and longitude between these ranges the data for latitude and longitude can be said to be of India, though we still can't be sure that each city's latitude and longitude is provided correctly or not. For this activity we will only calculate the outliers of lat and long out of India's boundary. If any of it is found out of India, we will fix the value for those using the existing mode of latitude/longitude for the city

Plotting last 600 records location of the Complete dataset

In [52]:
#Plotting top 600 records to start with
data_to_map = events_data_df.head(600)

map_India = plot_on_map(data_to_map)
map_India

In [53]:
#Plotting last 600 records to start with
data_to_map = events_data_df.tail(600)

map_India = plot_on_map(data_to_map)
map_India

Looking at above 2 Maps, it seems all the top and bottom records are in India, let's do the analysis based on Latitude and Longitude values

Let's first describe the latitude and longitude column values and see, if anything lies outside of the range of India's range of latitude and longitude

In [54]:
events_data_df[['latitude', 'longitude']].describe()

,latitude,longitude
count,3.252950e+06,3.252950e+06
mean,2.168856e+01,7.815869e+01
std,5.789083e+00,4.235501e+00
min,8.190110e+00,1.256700e+01
25%,1.780171e+01,7.583544e+01
50%,2.216548e+01,7.726814e+01
75%,2.868278e+01,8.031916e+01
max,4.187190e+01,9.545946e+01


Looking at the max and min values for latitude (max 41.87, greater than 37.60) and longitude (min 42.35, lesser than 68.7), looks like few records are out of the range of India. Let's do below analysis to find the places having locations outside of India

##### Sort columns by latitude to do analysis of outliers of latitude values

In [55]:
#Sorting by latitude in ascending order (default)
events_lat_sorted = events_data_df.sort_values('latitude')

In [56]:
#Getting top 300 records for plotting on the map
data_to_map = events_lat_sorted.head(300)

map_India = plot_on_map(data_to_map)
map_India

All records seems to be in India and looks like to be South of India, it confirms our analysis of all latitude records within 8.4

In [57]:
#Now plotting the last X records sorted by latitude
#Started with 300 and found few records outside of India, then after few retries found the last 42 records plotted out of India
data_to_map = events_lat_sorted.tail(42)

map_India = plot_on_map(data_to_map)
map_India

After few retries, below Observation can be confirmd:<br/>
Problem identified in latitude values at the higher end (42 records, value greater than 34.5553), for many cities the latitude and longitude values are not correct. This also confirms our inital analysis of latitude value more than 37.6

##### Sort columns by longitude to do analysis of outliers of longitude values

In [58]:
#Sorting by latitude in ascending order (default)
events_lat_sorted = events_data_df.sort_values('longitude')

In [59]:
#Now plotting the top X records sorted by latitude
#Started with 300 and found few records outside of India, then after few retries found the last 63 records plotted out of India
data_to_map = events_lat_sorted.head(63)

map_India = plot_on_map(data_to_map)
map_India

After few retries, below Observation can be confirmd:<br/>
Problem identified in longitude values (63 records, value less than 69.207500), for many cities the latitude and longitude values are not correct. This also confirms our inital analysis of longitude value less than 68.7

In [60]:
#Now plotting the last X records sorted by longitude
data_to_map = events_lat_sorted.tail(300)

map_India = plot_on_map(data_to_map)
map_India

All records seems to be in India and looks like to be East of India, it confirms our analysis of all longitude records within 95.49

### Summary of Latitude and longitude data analysis and fix the incorrect data

### Filtering events data by 6 states
Before this step we will need to remove/replace all the NaN values by the resective values

In [61]:
# Find Unique States in the events data 
pd.DataFrame(events_data_df.state.unique())[0].sort_values(ascending = False)

1                   WestBengal
24                 Uttaranchal
13                UttarPradesh
23                     Tripura
16                   Telangana
2                    TamilNadu
10                   Rajasthan
12                      Punjab
25                 Pondicherry
11                      Orissa
14                    Nagaland
22                     Mizoram
27                   Meghalaya
26                     Manipur
5                  Maharashtra
8                MadhyaPradesh
7                       Kerala
9                    Karnataka
3                    Jharkhand
19             JammuandKashmir
29             HimachalPradesh
15                     Haryana
6                      Gujarat
21                         Goa
0                        Delhi
17                Chhattisgarh
30                  Chandigarh
18                       Bihar
20                       Assam
28            ArunachalPradesh
4                AndhraPradesh
31    AndamanandNicobarIslands
Name: 0,

In [62]:
states_to_filter = ['AndhraPradesh', 'Pondicherry', 'Mizoram', 'AndamanandNicobarIslands', 'Meghalaya', 'HimachalPradesh']

In [63]:
boolean_series = events_data_df.state.isin(states_to_filter)
#boolean_series
events_data_filtered_df = events_data_df[boolean_series]
# Reset the indexes
events_data_filtered_df = events_data_filtered_df.reset_index(drop=True)
#events_data_filtered_df.head()
#events_data_filtered_df.state.unique()

# Checking the Unique values for each column
events_data_filtered_df.nunique()

event_id     329172
device_id      5223
timestamp    240679
longitude      5203
latitude       5210
city            113
state             6
dtype: int64

In [64]:
events_data_filtered_df.shape

(329172, 7)

In [65]:
events_data_filtered_df.head(5)

,event_id,device_id,timestamp,longitude,latitude,city,state
0,1078723,-512424213979236423,2016-05-02 02:21:20,83.398244,17.768149,Visakhapatnam,AndhraPradesh
1,280014,-8879643774076165575,2016-05-05 13:06:01,78.155397,16.390327,Wanparti,AndhraPradesh
2,2334601,-601883305299256779,2016-05-05 11:17:48,83.380111,17.828583,Visakhapatnam,AndhraPradesh
3,2064864,-2764520535330416860,2016-05-03 23:58:20,83.315014,17.825280,Visakhapatnam,AndhraPradesh
4,1341801,4986891477794016217,2016-05-07 15:24:58,83.324339,17.778384,Visakhapatnam,AndhraPradesh


In [66]:
missing_data(events_data_filtered_df)

,Total,Percent
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0
event_id,0,0.0


#### Merge the data of all three data sets

In [67]:
# Merging Gender Age and Phone Brand Model datasets on the basis of device id
# As gender age dataset contains lesser record, we are keeping it on left and joining left
merged_ga_pbm = pd.merge(gender_age_train_df, phone_brand_device_model_df, how ='left', left_on='device_id', right_on='device_id')

In [68]:
merged_ga_pbm.shape

(74645, 6)

In [69]:
missing_data(merged_ga_pbm)

,Total,Percent
model_name,28,0.037511
brand,28,0.037511
Age_Group,0,0.000000
Age,0,0.000000
Gender,0,0.000000
device_id,0,0.000000


###### As we analyzed above, there are 74617 common records between Gender Age and Phone datasets, which means the phone dataset does not contain 28 records, which are there in Gender Age dataset
Not looking to remove any of these missing records, first we will merge the event data also and then see if there are any missing records

In [70]:
# Changing the data type of device_id column in merged data set to String from int
# If we do not do this, the merge between events and this merged data set will not work as device_id in events dataset is string
merged_ga_pbm['device_id'] = merged_ga_pbm['device_id'].astype(str)

In [71]:
# Merging Filtered Events Data and above merged datasets on the basis of device id
# As events dataset contains lesser device id records, we are keeping it on left and joining left

final_df = pd.merge(events_data_filtered_df, merged_ga_pbm, how ='left', left_on='device_id', right_on='device_id')

In [72]:
final_df.shape

(329172, 12)

In [73]:
missing_data(final_df)

,Total,Percent
model_name,0,0.0
brand,0,0.0
Age_Group,0,0.0
Age,0,0.0
Gender,0,0.0
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0


In [74]:
final_df.nunique()

event_id      329172
device_id       5223
timestamp     240679
longitude       5203
latitude        5210
city             113
state              6
Gender             2
Age               72
Age_Group         12
brand             69
model_name       721
dtype: int64

In [75]:
pd.DataFrame(final_df.state.unique())[0].sort_values(ascending = False)

2                 Pondicherry
1                     Mizoram
3                   Meghalaya
4             HimachalPradesh
0               AndhraPradesh
5    AndamanandNicobarIslands
Name: 0, dtype: object

In [76]:
##### Create DF for each states to be printed on the map to see the correct lat and longitude

In [77]:
'''

ap_df = final_df.loc[final_df['state'] == 'AndhraPradesh']
ap_df.shape

hp_df = final_df.loc[final_df['state'] == 'HimachalPradesh']
hp_df.shape

miz_df = final_df.loc[final_df['state'] == 'Mizoram']
miz_df.shape

meg_df = final_df.loc[final_df['state'] == 'Meghalaya']
meg_df.shape

an_df = final_df.loc[final_df['state'] == 'AndamanandNicobarIslands']
an_df.shape

p_df = final_df.loc[final_df['state'] == 'Pondicherry']
p_df.shape

center = [p_df.iloc[0]['latitude'], p_df.iloc[0]['longitude']]
map_Pondicherry = folium.Map(location=center, zoom_start=4)

for index, phone_loc in p_df.iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_Pondicherry)
    
map_Pondicherry

center = [an_df.iloc[0]['latitude'], an_df.iloc[0]['longitude']]
map_AndamanNicobar = folium.Map(location=center, zoom_start=4)

for index, phone_loc in an_df.iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_AndamanNicobar)
    
map_AndamanNicobar

center = [meg_df.iloc[0]['latitude'], meg_df.iloc[0]['longitude']]
map_Meghalaya = folium.Map(location=center, zoom_start=4)

for index, phone_loc in meg_df.iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_Meghalaya)
    
map_Meghalaya

center = [miz_df.iloc[0]['latitude'], miz_df.iloc[0]['longitude']]
map_Mizoram = folium.Map(location=center, zoom_start=4)

for index, phone_loc in miz_df.iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_Mizoram)

map_Mizoram

center = [hp_df.iloc[0]['latitude'], hp_df.iloc[0]['longitude']]
map_HimachalPradesh = folium.Map(location=center, zoom_start=4)

for index, phone_loc in hp_df.iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_HimachalPradesh)
    
map_HimachalPradesh

center = [ap_df.iloc[0]['latitude'], ap_df.iloc[0]['longitude']]
map_AndhraPradesh = folium.Map(location=center, zoom_start=4)

for index, phone_loc in ap_df.head(4000).iterrows():
    #print(phone_loc['latitude'], franchise['longitude'])
    location = [phone_loc['latitude'], phone_loc['longitude']]
    folium.Marker(location).add_to(map_AndhraPradesh)

map_AndhraPradesh
'''

"\n\nap_df = final_df.loc[final_df['state'] == 'AndhraPradesh']\nap_df.shape\n\nhp_df = final_df.loc[final_df['state'] == 'HimachalPradesh']\nhp_df.shape\n\nmiz_df = final_df.loc[final_df['state'] == 'Mizoram']\nmiz_df.shape\n\nmeg_df = final_df.loc[final_df['state'] == 'Meghalaya']\nmeg_df.shape\n\nan_df = final_df.loc[final_df['state'] == 'AndamanandNicobarIslands']\nan_df.shape\n\np_df = final_df.loc[final_df['state'] == 'Pondicherry']\np_df.shape\n\ncenter = [p_df.iloc[0]['latitude'], p_df.iloc[0]['longitude']]\nmap_Pondicherry = folium.Map(location=center, zoom_start=4)\n\nfor index, phone_loc in p_df.iterrows():\n    #print(phone_loc['latitude'], franchise['longitude'])\n    location = [phone_loc['latitude'], phone_loc['longitude']]\n    folium.Marker(location).add_to(map_Pondicherry)\n    \nmap_Pondicherry\n\ncenter = [an_df.iloc[0]['latitude'], an_df.iloc[0]['longitude']]\nmap_AndamanNicobar = folium.Map(location=center, zoom_start=4)\n\nfor index, phone_loc in an_df.iterrows()